In [2]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import itertools
import operator
import collections
import random
import functools
import statistics
import math
import regex
import pprint as pp
import networkx as nx


plt.rc('image', cmap='viridis')
plt.style.use('dark_background')


def Input(day):
    'Open input file for the day'
    file = '2023/input{0}.txt'.format(day)
    return open(file)

def strip_lines(f):
    return [l.strip() for l in f.readlines()] 


## Day 1: Trebuchet?!

In [3]:
with Input(1) as f:
    lines = strip_lines(f)

def calibrate_one(line):
    digits = regex.findall('\d', line)
    return int(digits[0]) * 10 + int(digits[-1])

def calibrate_two(line):
    digits = regex.findall('\d|one|two|three|four|five|six|seven|eight|nine', line, overlapped=True)
    return word_to_int(digits[0]) * 10 + word_to_int(digits[-1])

def word_to_int(word):
    match word:
        case 'one':
            return 1
        case 'two':
            return 2
        case 'three':
            return 3
        case 'four':
            return 4
        case 'five':
            return 5
        case 'six':
            return 6
        case 'seven':
            return 7
        case 'eight':
            return 8
        case 'nine':
            return 9
        case _:
            return int(word)

print('Part One: ', sum(map(calibrate_one, lines)))
print('Part Two: ', sum(map(calibrate_two, lines)))


Part One:  55172
Part Two:  54925


# Day 2: Cube Conundrum

In [4]:
with Input(2) as f:
    lines = strip_lines(f)

def parse_game(line):
    ''' 
    Find all the numbers for each color and return a tuple
    of ([red], [green], [blue], game_number)
    '''
    game_number = int(regex.match(r'Game (\d+)', line)[1])
    red = list(map(int, regex.findall(r'(\d+) red', line)))
    green = list(map(int, regex.findall(r'(\d+) green', line)))
    blue = list(map(int, regex.findall(r'(\d+) blue', line)))
    game = (red, green, blue, game_number)
    return game


def possible(game):
    #12 red cubes, 13 green cubes, and 14 blue cubes
    red = all([True if red <= 12 else False for red in game[0]])
    green = all([True if green <= 13 else False for green in game[1]])
    blue = all([True if blue <= 14 else False for blue in game[2]])
    return red and green and blue

games = [parse_game(line) for line in lines]

print('Part One: ', sum([game[3] for game in games if possible(game)]))
print('Part Two: ', sum(max(game[0]) * max(game[1]) * max(game[2]) for game in games))


Part One:  2278
Part Two:  67953


# Day 3: Gear Ratios

In [5]:
with Input(3) as f:
    schematic = strip_lines(f)

MAX = len(schematic)

# vectors for adjacent cells
neighbor_indexes = {(dy, dx) for dx,dy in itertools.permutations([-1,-1,0,1,1], 2)}

def find_potential_parts(schematic):
    '''
    Find the x ranges of all parts, indexed by y
    [[part, ...], ...]
    '''
    return [ list(regex.finditer('\d+', line)) for line in schematic]

def neighbors(schematic):
    '''
    Return the value of all neighboring cells
    '''
    return lambda i : [schematic[i[0] + dy][i[1] + dx] for dy,dx in neighbor_indexes if i[0]+ dy >= 0 and i[0]+ dy < MAX and i[1]+ dx >= 0 and i[1]+ dx < MAX]

def neighbor_locations(schematic):
    '''
    Return the indexes of all neighboring cells
    '''
    return lambda i : [(i[0] + dy, i[1] + dx) for dy,dx in neighbor_indexes if i[0]+ dy >= 0 and i[0]+ dy < MAX and i[1]+ dx >= 0 and i[1]+ dx < MAX]

get_neighbors = neighbors(schematic)
get_neighbor_indexes = neighbor_locations(schematic)

def valid_part(y, part):
    '''
    Does the part a y, part_x_range neighbor a symbol?
    '''
    part_range = range(part.start(), part.end())

    for x in part_range:
        neighbors =  get_neighbors((y, x))
        for neighbor in neighbors:
            if neighbor not in '0123456789.':
                return True
    return False

def find_potential_gears(schematic):
    '''
    Find the (y,x) coordinates of all * characters
    '''
    return [(y,x) for y, line in enumerate(schematic) for x, i in enumerate(line) if i == '*']

def find_neighboring_part_numbers(gear, parts_list):
    '''
    Find all the part numbers that neighbor a potential gear, valid gears will only have two neighboring parts
    '''
    neighboring_part_numbers = []
    gear_neighbors = get_neighbor_indexes(gear)
    for y, parts in enumerate(parts_list):
        for part in parts:
            part_range = range(part.start(), part.end())
            for x in part_range:
                if (y,x) in gear_neighbors:
                    neighboring_part_numbers.append(int(part.group()))
                    break
    return neighboring_part_numbers

# A bunch of junk follows

potential_parts = find_potential_parts(schematic)
valid_parts = [[part for part in part_list if valid_part(y, part)] for y, part_list in enumerate(potential_parts)]

valid_part_sum = sum([int(part.group()) for parts in valid_parts for part in parts])

potential_gears = find_potential_gears(schematic)
potential_gear_neighboring_part_numbers = [find_neighboring_part_numbers(gear, valid_parts) for gear in potential_gears]

valid_gear_neighboring_part_numbers = list(filter(lambda ls: len(ls) == 2, potential_gear_neighboring_part_numbers))

gear_values = [functools.reduce(operator.mul, part_numbers, 1) for part_numbers in valid_gear_neighboring_part_numbers]
valid_gear_sum = sum(gear_values)

print('Part One: ', valid_part_sum)
print('Part Two: ', valid_gear_sum)


Part One:  509115
Part Two:  75220503


# Day 4: Scratchcards 

In [130]:
with Input(4) as f:
    lines = strip_lines(f)

Card = collections.namedtuple('Card', ['card_number', 'winning_numbers', 'numbers_you_have', 'value', 'matches', 'copies'])

def parse_card(line):
    '''
    Turn a line into a Card
    '''
    card_number = int(regex.match(r'Card +(\d+):', line).group(1))
    winning_numbers = set(map(int, regex.findall(r':(.*)\|', line)[0].split()))
    numbers_you_have = set(map(int, regex.findall(r'\|(.*)', line)[0].split()))

    return Card(card_number, winning_numbers, numbers_you_have, 0, 0, 1)

def assess_value(card):
    '''
    Calculate the value of a card, as well as the number of matches
    '''
    matches = len(card.winning_numbers.intersection(card.numbers_you_have))
    winner = 1 if matches != 0 else 0
    return card._replace(value=2 ** (matches - 1) * winner, matches= matches)


# parse all the cards
cards = [parse_card(line) for line in lines]

#value all the cards
valued_cards = list(map(assess_value, cards))

# sum the values of the cards
part_one = sum([card.value for card in valued_cards])

# for each card, add to cards # of copies as necessary
for i, card in enumerate(valued_cards):
    # add this cards number of copies to subsequent cards based on the number of matches
    for idx in range(i+1, i+1+card.matches):
        valued_cards[idx] = valued_cards[idx]._replace(copies=valued_cards[idx].copies+card.copies)

part_two = sum([card.copies for card in valued_cards])


print('Part One: ', int(part_one))
print('Part Two: ', part_two)


Part One:  21088
Part Two:  6874754
